In [22]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file


In [23]:
client : OpenAI = OpenAI()


In [24]:
from openai.types.beta import Assistant

# Upload a file with an "assistants" purpose
file = client.files.create(
  file=open("zia_profile.pdf", "rb"),
  purpose='assistants'
)

print(file)

FileObject(id='file-NO4t4ESmDXWTZShIixbv1e0O', bytes=48802, created_at=1705153439, filename='zia_profile.pdf', object='file', purpose='assistants', status='processed', status_details=None)


In [25]:
assistant: Assistant = client.beta.assistants.create(
  name="Student Support Assistant",
  instructions="You are a student support chatbot. Use your knowledge base to best respond to student queries about Zia U. Khan.",
  model="gpt-3.5-turbo-1106",
  tools=[{"type": "retrieval"}],
  file_ids=['file-V1XIkxQnsbMtD7kkkUrdtoMS',]
)


# Step 2: Create a Thread


In [26]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

print(thread)


Thread(id='thread_z1zXPqDDVWLXd2DkG7T14kn3', created_at=1705153441, metadata={}, object='thread')


# Step 3: Add a Message to a Thread


In [21]:
from openai.types.beta.threads.thread_message import ThreadMessage

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="When and which city Zia U. Khan was born?"
)


# Step 4: Run the Assistant


In [27]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Pakistani. The user is the student of PIAIC."
)


# Step 5: Check the Run status


In [28]:
run: Run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

print(run)


Run(id='run_TBemiVu8UEO5oOTM1XuA6m2x', assistant_id='asst_rVHKUOo5FDKy9dxi85w40ezI', cancelled_at=None, completed_at=1705153478, created_at=1705153467, expires_at=None, failed_at=None, file_ids=['file-V1XIkxQnsbMtD7kkkUrdtoMS'], instructions='Please address the user as Pakistani. The user is the student of PIAIC.', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1705153467, status='completed', thread_id='thread_z1zXPqDDVWLXd2DkG7T14kn3', tools=[ToolAssistantToolsRetrieval(type='retrieval')])


# Step 6: Display the Assistant's Response


In [29]:
# from openai.resources.beta.threads.messages.messages import SyncCursorPage 

messages: list[ThreadMessage] = client.beta.threads.messages.list(
  thread_id=thread.id
)

for m in reversed(messages.data):
  print(m.role + ": " + m.content[0].text.value)


assistant: The document provides a detailed account of Mr. Zia U. Khan's journey and his efforts towards nation-building through education and technology. It outlines his educational achievements, professional experience in the USA, and his return to Pakistan in the late 1990s to initiate a nation-building movement focused on education and technology.

It mentions the formation of the "Operation Bader" movement, which aimed to provide low-cost education and training in technology, initially gaining significant traction but facing challenges after 9/11 due to changes in the international business environment. Despite these challenges, the movement persevered and evolved, eventually gaining momentum again with the rise of freelancing opportunities and the involvement of other institutions such as the Saylani Welfare Trust.

Moreover, the document highlights the establishment of the Presidential Initiative for Artificial Intelligence and Computing (PIAIC) under the leadership of Dr. Arif 